# A2: Logistic Regression (AQ-10 + Demographic Features)

This notebook implements a logistic regression model using AQ-10 questionnaire responses together with basic demographic features.

**Purpose:**  
To examine whether including demographic information provides additional predictive value beyond AQ-10 features alone.


In [21]:
from pathlib import Path
import sys

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

In [23]:
# 0) PATHS
NOTEBOOK_DIR = Path.cwd() 
PROJECT_ROOT = NOTEBOOK_DIR.parent      

DATA_DIR = PROJECT_ROOT / "data"
SRC_DIR = PROJECT_ROOT / "src"
RESULTS_DIR = PROJECT_ROOT / "results" / "A2"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

sys.path.insert(0, str(SRC_DIR))

from common_setup import (
    load_arff,
    prepare_df_AQ_DEMO,
    build_preprocessor,
    run_experiment
)

ADULT_ARFF_PATH = DATA_DIR / "Autism-Adult-Data.arff"
CHILD_ARFF_PATH = DATA_DIR / "Autism-Child-Data.arff"

In [25]:
# LOAD DATA (ARFF)
adult_raw = load_arff(ADULT_ARFF_PATH)
child_raw = load_arff(CHILD_ARFF_PATH)

print("\nLoaded adult shape:", adult_raw.shape)
print("Loaded child shape:", child_raw.shape)



Loaded adult shape: (704, 21)
Loaded child shape: (292, 21)


In [27]:
# PREP FEATURES (AQ + DEMO)

Xa, ya, num_adult, cat_adult, feat_adult = prepare_df_AQ_DEMO(adult_raw, "Adult")
Xc, yc, num_child, cat_child, feat_child = prepare_df_AQ_DEMO(child_raw, "Child")




[Adult] AQ columns: ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']
[Adult] Dropping leaky column 'result'
[Adult] Using label column: 'Class/ASD'
[Adult] X shape: (704, 19)
[Adult] y counts: [515 189]

[Child] AQ columns: ['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score']
[Child] Dropping leaky column 'result'
[Child] Using label column: 'Class/ASD'
[Child] X shape: (292, 19)
[Child] y counts: [151 141]


In [29]:
# PIPELINE (Standard Logistic Regression)

pre_adult = build_preprocessor(num_adult, cat_adult)
pre_child = build_preprocessor(num_child, cat_child)

a2_clf = LogisticRegression(
    max_iter=3000,
    solver="lbfgs"
)

a2_adult_pipe = Pipeline([
    ("prep", pre_adult),
    ("clf", a2_clf)
])

a2_child_pipe = Pipeline([
    ("prep", pre_child),
    ("clf", a2_clf)
])

In [31]:
# RUN 5-FOLD CV

adult_folds, adult_summary = run_experiment(
    Xa, ya, a2_adult_pipe,
    exp_name="A2 Adult (LR AQ+Demo)",
    n_splits=5,
    random_state=42
)

child_folds, child_summary = run_experiment(
    Xc, yc, a2_child_pipe,
    exp_name="A2 Child (LR AQ+Demo)",
    n_splits=5,
    random_state=42
)

[A2 Adult (LR AQ+Demo)] Fold 1: acc=1.0000, prec=1.0000, rec=1.0000, f1=1.0000, roc_auc=1.0000, pr_auc=1.0000
[A2 Adult (LR AQ+Demo)] Fold 2: acc=1.0000, prec=1.0000, rec=1.0000, f1=1.0000, roc_auc=1.0000, pr_auc=1.0000
[A2 Adult (LR AQ+Demo)] Fold 3: acc=1.0000, prec=1.0000, rec=1.0000, f1=1.0000, roc_auc=1.0000, pr_auc=1.0000
[A2 Adult (LR AQ+Demo)] Fold 4: acc=1.0000, prec=1.0000, rec=1.0000, f1=1.0000, roc_auc=1.0000, pr_auc=1.0000
[A2 Adult (LR AQ+Demo)] Fold 5: acc=1.0000, prec=1.0000, rec=1.0000, f1=1.0000, roc_auc=1.0000, pr_auc=1.0000

[A2 Adult (LR AQ+Demo)] 5-fold mean metrics:
  accuracy: 1.0000
  precision: 1.0000
  recall: 1.0000
  f1: 1.0000
  roc_auc: 1.0000
  pr_auc: 1.0000
[A2 Child (LR AQ+Demo)] Fold 1: acc=1.0000, prec=1.0000, rec=1.0000, f1=1.0000, roc_auc=1.0000, pr_auc=1.0000
[A2 Child (LR AQ+Demo)] Fold 2: acc=1.0000, prec=1.0000, rec=1.0000, f1=1.0000, roc_auc=1.0000, pr_auc=1.0000
[A2 Child (LR AQ+Demo)] Fold 3: acc=1.0000, prec=1.0000, rec=1.0000, f1=1.0000, 

In [33]:
# SAVE OUTPUTS

adult_folds.to_csv(RESULTS_DIR / "A2_adult_folds.csv", index=False)
adult_summary.to_csv(RESULTS_DIR / "A2_adult_summary.csv", index=False)

child_folds.to_csv(RESULTS_DIR / "A2_child_folds.csv", index=False)
child_summary.to_csv(RESULTS_DIR / "A2_child_summary.csv", index=False)

print("\nSaved A2 results to:", RESULTS_DIR)


Saved A2 results to: C:\Users\14ush\Desktop\asc-screening-xai-dissertation\results\A2


In [35]:
# SHUFFLED-LABEL SANITY CHECK

def shuffled_label_run(X, y, pipe, name, random_state=42):
    y_shuf = shuffle(y, random_state=random_state)
    _, summary = run_experiment(
        X, y_shuf, pipe,
        exp_name=f"{name} (SHUFFLED LABELS)",
        n_splits=5,
        random_state=random_state
    )
    return summary

adult_shuf_summary = shuffled_label_run(Xa, ya, a2_adult_pipe, "A2 Adult", random_state=42)
child_shuf_summary = shuffled_label_run(Xc, yc, a2_child_pipe, "A2 Child", random_state=42)

adult_shuf_summary.to_csv(RESULTS_DIR / "A2_adult_shuffled_summary.csv", index=False)
child_shuf_summary.to_csv(RESULTS_DIR / "A2_child_shuffled_summary.csv", index=False)

print("\nSaved shuffled-label summaries to:", RESULTS_DIR)

print("\nA2 Adult mean metrics:")
display(adult_summary)

print("\nA2 Child mean metrics:")
display(child_summary)

print("\nA2 Adult shuffled-label mean metrics:")
display(adult_shuf_summary)

print("\nA2 Child shuffled-label mean metrics:")
display(child_shuf_summary)

[A2 Adult (SHUFFLED LABELS)] Fold 1: acc=0.7021, prec=0.1667, rec=0.0263, f1=0.0455, roc_auc=0.5286, pr_auc=0.2896
[A2 Adult (SHUFFLED LABELS)] Fold 2: acc=0.7376, prec=0.6667, rec=0.0526, f1=0.0976, roc_auc=0.5388, pr_auc=0.3468
[A2 Adult (SHUFFLED LABELS)] Fold 3: acc=0.7234, prec=0.3333, rec=0.0263, f1=0.0488, roc_auc=0.5026, pr_auc=0.2899
[A2 Adult (SHUFFLED LABELS)] Fold 4: acc=0.7163, prec=0.2500, rec=0.0263, f1=0.0476, roc_auc=0.5245, pr_auc=0.2815
[A2 Adult (SHUFFLED LABELS)] Fold 5: acc=0.7071, prec=0.0000, rec=0.0000, f1=0.0000, roc_auc=0.5017, pr_auc=0.2824

[A2 Adult (SHUFFLED LABELS)] 5-fold mean metrics:
  accuracy: 0.7173
  precision: 0.2833
  recall: 0.0263
  f1: 0.0479
  roc_auc: 0.5192
  pr_auc: 0.2980
[A2 Child (SHUFFLED LABELS)] Fold 1: acc=0.5085, prec=0.4783, rec=0.3929, f1=0.4314, roc_auc=0.4055, pr_auc=0.4309
[A2 Child (SHUFFLED LABELS)] Fold 2: acc=0.4407, prec=0.4286, rec=0.4138, f1=0.4211, roc_auc=0.4460, pr_auc=0.4661
[A2 Child (SHUFFLED LABELS)] Fold 3: acc

,accuracy,precision,recall,f1,roc_auc,pr_auc
0,1.0,1.0,1.0,1.0,1.0,1.0



A2 Child mean metrics:


,accuracy,precision,recall,f1,roc_auc,pr_auc
0,1.0,1.0,1.0,1.0,1.0,1.0



A2 Adult shuffled-label mean metrics:


,accuracy,precision,recall,f1,roc_auc,pr_auc
0,0.717315,0.283333,0.026316,0.047883,0.519248,0.298039



A2 Child shuffled-label mean metrics:


,accuracy,precision,recall,f1,roc_auc,pr_auc
0,0.489831,0.472276,0.468473,0.46859,0.452563,0.460124
